# ATEK Demo 1: ATEK data preprocessing -> visualization -> loading -> inference

In [1]:
import faulthandler

import logging
import os
from logging import StreamHandler
import numpy as np
import torch
import sys
from tqdm import tqdm

from atek.data_preprocess.atek_wds_writer import AtekWdsWriter

from atek.data_preprocess.sample_builders.obb_sample_builder import ObbSampleBuilder
from atek.data_preprocess.subsampling_lib.temporal_subsampler import (
    CameraTemporalSubsampler,
)

from atek.data_loaders.atek_wds_dataloader import (
    create_native_atek_dataloader
)

from atek.data_loaders.cubercnn_model_adaptor import (
    create_atek_dataloader_as_cubercnn
)
from atek.data_preprocess.atek_data_sample import (
    create_atek_data_sample_from_flatten_dict,
)
from atek.viz.atek_visualizer import NativeAtekSampleVisualizer

from omegaconf import OmegaConf

faulthandler.enable()

# Configure logging to display the log messages in the notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

logger = logging.getLogger()

### Helper functions used in this script

In [2]:
def print_data_sample_dict_content(data_sample):
    """
    A helper function to print the content of data sample dict
    """
    for key, val in data_sample.items():
        msg = f"{key}: is a {type(val)}, "
        if isinstance(val, torch.Tensor):
            msg += f"with shape of : {val.shape}"
        elif isinstance(val, list):
            msg += f"with len of : {len(val)}"
        elif isinstance(val, str):
            msg += f"value is {val}"
        else:
            pass
        logger.info(msg)

## Set up data and code paths

In [5]:
# Follow the following guide to download example ADT sequence to a local path `~/Documents/projectaria_tools_adt_data`
# https://facebookresearch.github.io/projectaria_tools/docs/open_datasets/aria_digital_twin_dataset/dataset_download. 

# Set up local data paths
data_dir = os.path.join(os.path.expanduser("~"), "Documents", "projectaria_tools_adt_data")
sequence_name = "Apartment_release_golden_skeleton_seq100_10s_sample_M1292"
example_adt_data_dir = os.path.join(data_dir, sequence_name)
output_wds_path = os.path.join(data_dir, "wds_output")

# Set up ATEK paths
atek_src_path = os.path.join(os.path.expanduser("~"), "atek_on_fbsource")
atek_preprocess_config_path = os.path.join(atek_src_path, "atek", "configs", "obb_preprocess_base.yaml")
atek_viz_config_path = os.path.join(atek_src_path, "atek", "configs", "obb_viz.yaml")
category_mapping_file = os.path.join(atek_src_path, "data", "adt_prototype_to_atek.csv")

# Set up trained model weight path
# model_ckpt_path = os.path.join()

# Example 1: ATEK data preprocessing

### Step 2: Set up ATEK data preprocessors

In [5]:
# Set up a visualizer to visualize the preprocessed samples, on the fly
viz_conf = OmegaConf.load(atek_viz_config_path)
atek_visualizer = NativeAtekSampleVisualizer(viz_prefix = "preprocess_visualizer", conf = viz_conf)

[2024-08-28T21:50:46Z INFO  re_sdk_comms::server] Hosting a SDK server over TCP at 0.0.0.0:9876. Connect with the Rerun logging SDK.
[2024-08-28T21:50:47Z INFO  winit::platform_impl::platform::x11::window] Guessed window scale factor: 1.0999959309895833
[2024-08-28T21:50:47Z INFO  re_sdk_comms::server] New SDK client connected from: 127.0.0.1:42902
[2024-08-28T21:50:47Z WARN  wgpu_hal::gles::egl] No config found!
[2024-08-28T21:50:47Z WARN  wgpu_hal::gles::egl] EGL says it can present to the window but not natively
[2024-08-28T21:50:47Z INFO  egui_wgpu] There were 3 available wgpu adapters: {backend: Vulkan, device_type: DiscreteGpu, name: "NVIDIA GeForce RTX 3080", driver: "NVIDIA", driver_info: "550.76", vendor: 0x10DE, device: 0x2216}, {backend: Vulkan, device_type: Cpu, name: "llvmpipe (LLVM 16.0.6, 256 bits)", driver: "llvmpipe", driver_info: "Mesa 23.1.9 (LLVM 16.0.6)", vendor: 0x10005}, {backend: Gl, device_type: Other, name: "NVIDIA GeForce RTX 3080/PCIe/SSE2", driver: "OpenGL"

In [6]:
# Create object detection sample builder
preprocess_conf = OmegaConf.load(atek_preprocess_config_path)
atek_preprocessor = create_atek_preprocessor_from_conf(conf = preprocess_conf, raw_data_folder, output_wds_path, viz_conf = viz_conf)
atek_preprocessor.process_all_data()
    


sample_builder = ObbSampleBuilder(
    conf=conf.processors,
    vrs_file=os.path.join(example_adt_data_dir, "video.vrs"),
    sequence_name=sequence_name,
    mps_files={
        "mps_closedloop_traj_file": os.path.join(
            example_adt_data_dir, "aria_trajectory.csv"
        ),
    },
    gt_files={
        "obb3_file": os.path.join(example_adt_data_dir, "3d_bounding_box.csv"),
        "obb3_traj_file": os.path.join(example_adt_data_dir, "scene_objects.csv"),
        "obb2_file": os.path.join(example_adt_data_dir, "2d_bounding_box.csv"),
        "instance_json_file": os.path.join(example_adt_data_dir, "instances.json"),
        "category_mapping_file": category_mapping_file,
    },
)

# From config, create a class to subsample data
subsampler = CameraTemporalSubsampler(
    vrs_file=os.path.join(example_adt_data_dir, "video.vrs"),
    conf=conf.camera_temporal_subsampler,
)

# Create WebDataset (WDS) writer to write preprocessed data to local
atek_wds_writer = AtekWdsWriter(
    output_path=output_wds_path,
    conf=conf.wds_writer,
)



[ProgressLogger][INFO]: 2024-08-28 14:50:48: Opening /home/louy/Documents/projectaria_tools_adt_data/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/video.vrs...
[MultiRecordFileReader][DEBUG]: Opened file '/home/louy/Documents/projectaria_tools_adt_data/Apartment_release_golden_skeleton_seq100_10s_sample_M1292/video.vrs' and assigned to reader #0
[VrsDataProvider][INFO]: streamId 211-1/camera-et activated
[VrsDataProvider][INFO]: streamId 214-1/camera-rgb activated
[VrsDataProvider][INFO]: streamId 247-1/baro0 activated
[VrsDataProvider][WARNING]: Unsupported TimeSync mode: APP, ignoring.
[VrsDataProvider][INFO]: Timecode stream found: 285-2
[VrsDataProvider][INFO]: streamId 1201-1/camera-slam-left activated
[VrsDataProvider][INFO]: streamId 1201-2/camera-slam-right activated
[VrsDataProvider][INFO]: streamId 1202-1/imu-right activated
[VrsDataProvider][INFO]: streamId 1202-2/imu-left activated
[VrsDataProvider][INFO]: streamId 1203-1/mag0 activated
[ProgressLogger][INFO]: 2

## Step 3: preprocess data samples, visualize them, and write to WDS files

In [7]:
# Loop over all samples, and write valid ones to local tar files.
for i in range(subsampler.get_total_num_samples()):
    timestamps_ns = subsampler.get_timestamps_by_sample_index(i)

    for t in timestamps_ns:
        sample = sample_builder.get_sample_by_timestamp_ns(t)
        if sample is not None:
            atek_visualizer.plot_atek_sample(sample)
            atek_wds_writer.add_sample(data_sample=sample)

atek_wds_writer.close()
logger.info(f"finish writing WDS in {output_wds_path}")

# also save visualization file locally
atek_visualizer.save_viz(rrd_output_path = os.path.join(data_dir, "example_atek_preprocess_viz_2.rrd"))

Loaded #closed loop trajectory poses records: 300
# writing /home/louy/Documents/projectaria_tools_adt_data/wds_output/shards-0000.tar 0 0.0 GB 0
# writing /home/louy/Documents/projectaria_tools_adt_data/wds_output/shards-0001.tar 32 0.0 GB 32
# writing /home/louy/Documents/projectaria_tools_adt_data/wds_output/shards-0002.tar 32 0.0 GB 64
# writing /home/louy/Documents/projectaria_tools_adt_data/wds_output/shards-0003.tar 32 0.0 GB 96
2024-08-28 14:51:23,524 - INFO - finish writing WDS in /home/louy/Documents/projectaria_tools_adt_data/wds_output
2024-08-28 14:51:23,525 - INFO - Saving visualization to /home/louy/Documents/projectaria_tools_adt_data/example_atek_preprocess_viz_2.rrd


# Example 2: load ATEK WDS files back as Pytorch Dataset

## Step 1: Load ATEK WDS files natively, and visualize (as ATEK data sample dicts)

In [6]:
# Load Native ATEK WDS data
logger.info("-------------------- loading ATEK data natively --------------- ")

# Loading local WDS files
tar_file_urls = [os.path.join(output_wds_path, f"shards-000{i}.tar") for i in range(4)]

# Batch size is None so that no collation is invoked
atek_dataloader = create_native_atek_dataloader(urls = tar_file_urls, batch_size=None, repeat_flag=False)

# Loop over all samples in DataLoader and visualize
viz_conf = OmegaConf.load(atek_viz_config_path)
atek_visualizer = NativeAtekSampleVisualizer(viz_prefix = "dataloading_visualizer", conf = viz_conf)
for atek_sample_dict in atek_dataloader:
    # First convert it back to ATEK data sample and visualize
    atek_sample = create_atek_data_sample_from_flatten_dict(atek_sample_dict)
    atek_visualizer.plot_atek_sample(atek_sample)

# Print the content of the first sample in the DataLoader
logger.info("Loading native atek data sample as a Dict, containing the following: ")
first_sample = next(iter(atek_dataloader))
print_data_sample_dict_content(first_sample)


2024-08-28 18:10:27,597 - INFO - -------------------- loading ATEK data natively --------------- 


[2024-08-29T01:10:27Z INFO  re_sdk_comms::server] Hosting a SDK server over TCP at 0.0.0.0:9876. Connect with the Rerun logging SDK.
[2024-08-29T01:10:27Z INFO  winit::platform_impl::platform::x11::window] Guessed window scale factor: 1.0999959309895833
[2024-08-29T01:10:27Z INFO  re_sdk_comms::server] New SDK client connected from: 127.0.0.1:59324
[2024-08-29T01:10:27Z WARN  wgpu_hal::gles::egl] No config found!
[2024-08-29T01:10:27Z WARN  wgpu_hal::gles::egl] EGL says it can present to the window but not natively
[2024-08-29T01:10:27Z INFO  egui_wgpu] There were 3 available wgpu adapters: {backend: Vulkan, device_type: DiscreteGpu, name: "NVIDIA GeForce RTX 3080", driver: "NVIDIA", driver_info: "550.76", vendor: 0x10DE, device: 0x2216}, {backend: Vulkan, device_type: Cpu, name: "llvmpipe (LLVM 16.0.6, 256 bits)", driver: "llvmpipe", driver_info: "Mesa 23.1.9 (LLVM 16.0.6)", vendor: 0x10005}, {backend: Gl, device_type: Other, name: "NVIDIA GeForce RTX 3080/PCIe/SSE2", driver: "OpenGL"

2024-08-28 18:10:44,195 - INFO - Loading native atek data sample as a Dict, containing the following: 
2024-08-28 18:10:44,251 - INFO - __key__: is a <class 'str'>, value is _AtekDataSample_000000
2024-08-28 18:10:44,252 - INFO - __url__: is a <class 'str'>, value is /home/louy/Documents/projectaria_tools_adt_data/wds_output/shards-0000.tar
2024-08-28 18:10:44,252 - INFO - gt_data: is a <class 'dict'>, 
2024-08-28 18:10:44,253 - INFO - mfcd#camera-rgb+camera_label: is a <class 'str'>, value is camera-rgb
2024-08-28 18:10:44,253 - INFO - mfcd#camera-rgb+camera_model_name: is a <class 'str'>, value is CameraModelType.FISHEYE624
2024-08-28 18:10:44,254 - INFO - mfcd#camera-rgb+capture_timestamps_ns: is a <class 'torch.Tensor'>, with shape of : torch.Size([1])
2024-08-28 18:10:44,254 - INFO - mfcd#camera-rgb+exposure_durations_s: is a <class 'torch.Tensor'>, with shape of : torch.Size([1])
2024-08-28 18:10:44,255 - INFO - mfcd#camera-rgb+frame_ids: is a <class 'torch.Tensor'>, with shape o

## Step 2: load ATEK WDS data as CubeRCNN-type data and print content (using ModelAdaptor)

In [9]:
logger.info(
    "-------------------- ATEK WDS data can also be loaded as CubeRCNN --------------- "
)
cubercnn_dataloader = create_atek_dataloader_as_cubercnn(urls = tar_file_urls, batch_size = 1, repeat_flag = False)


# CubeRCNN's collation function will batch samples directly into list
logger.info("Loading atek data sample as a CubeRCNN-type data dict, containing the following: ")
cubercnn_sample_dict_list = next(iter(cubercnn_dataloader))
print_data_sample_dict_content(cubercnn_sample_dict_list[0])



2024-08-28 18:12:19,175 - INFO - -------------------- ATEK WDS data can also be loaded as CubeRCNN --------------- 
2024-08-28 18:12:19,177 - INFO - Loading atek data sample as a CubeRCNN-type data dict, containing the following: 


AssertionError: Only linear camera model supported in CubeRCNN model, this data has CameraModelType.FISHEYE624 instead.

# Example 3: Run Object detection inference using trained CubeRCNN model

## Step 1: load trained CubeRCNN model weights